 ## Integrantes
* Aline Cristini - 183132
* Camila Rodrigues - 183143
* Rafael Gimenes Leite - 101634


## Introdução
Esta análise tem como o objetivo do nosso exercício referente à aula 2 será simular um processo de seleção de um cientista de dados.
O Dataset selecionado pelo grupo é o **911**.

## Importando bibliotecas necessárias
Abordando o que nos foi apresentado até o presente momento no curso iremos interagir com o dataset utilizando  pandas, numpy, matplotlib entre outros.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib.backends.backend_agg import FigureCanvas
from matplotlib.figure import Figure

# definindo o tamanho da figura
fig = Figure(figsize=(7,7))

# definindo qual canvas utilizar 
canvas = FigureCanvas(fig)


In [ ]:
## leitura do csv para dataframe
df = pd.read_csv('../input/911.csv', delimiter=',')

In [ ]:
## conhecendo o dataset
df.info()

### Resumo das variáveis
* lat: Variável String, Latitude 
* lng: Variável String, Longitude
* desc: Variável String, Descrição da Chamada de Emergência
* zip: Variável String, CEP
* título: Variável String, Título
* timeStamp: Variável String, AAAA-MM-DD HH: MM: SS  (**talvez tenhamos que converter pra tipo data**)
* twp: Variável String, Township
* addr: Variável String, Endereço
* e: Variável String, variável Dummy (sempre 1)

In [ ]:
### removendo coluna e que não server para nada.
del df["e"]

In [ ]:
### conhecendo dados Nulos
df.isnull().sum()

### Convertendo campo Data, criando novos campos

In [ ]:
### convertendo pra date time a coluna timeStamp
df['timeStamp']=df['timeStamp'].apply(pd.to_datetime)
### criando mais 3 colunas com hora mes e ano pra facilitar nas estatisticas
df['hour']=df['timeStamp'].dt.hour
df['month']=df['timeStamp'].dt.month
df['dayOfWeekNumber']=df['timeStamp'].dt.dayofweek
### Mapeando o dayofweek em String
Dicdayofweek={0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri' , 5:'Sat', 6:'Sun'}
df['dayOfWeekStr'] = df['timeStamp'].dt.dayofweek.map(Dicdayofweek)
### Mapeando o Month em String
df['monthStr'] = df['timeStamp'].dt.strftime("%B")
### criando coluna Reason e Sub-Reason para ser utilizada nas estatísticas
# Emergency Medical Services
df['reason'] = df['title'].str.split(':').str.get(0)
df['subReason'] = df['title'].str.split(':').str.get(1)
df.head()

### Classificação das variáveis

In [ ]:
#Criando DataFrame com a Classificação das Variáveis
variaveis = [["lat", "Numérica Contínua"],["lng", "Numérica Contínua"],["desc", "Categórica Nominal"],["zip", "Numérica Contínua"],["title", "Categórica Nominal"]
             ,["timeStamp", "Numérica Contínua"],["twp", "Categórica Nominal"],["addr", "Categórica Nominal"],["Hour", "Numérica Discreta"],["Month", "Numérica Discreta"]
             ,["dayOfWeekNumber", "Numérica Discreta"],["dayOfWeekStr", "Categórica Ordinal"],["Reason", "Categórica Nominal"],["Sub-Reason", "Categórica Nominal"]
             ,["monthStr", "Categórica Ordinal"]]
variaveis = pd.DataFrame(variaveis, columns=["Variavel", "Classificação"])
variaveis

### Proporção das razões

In [ ]:
categoriaRazoes =  df['reason'].value_counts()
label = ['EMS','TRAFFIC','FIRE']
explode = (0.1, 0, 0)
fig1, ax1 = plt.subplots()
ax1.pie(categoriaRazoes, explode=explode,labels = label, autopct='%1.1f%%',shadow=True, startangle=100)
#categoriaRazoes.plot.pie(figsize=(10, 8),autopct='%1.1f%%', startangle=180,fontsize=14)
plt.title('Chamados por Motivos')
ax1.axis('equal') 
plt.show()

### Top 10 maiores razões de chamadas gerais

In [ ]:
top10TodasRazoes2 = df['subReason'].value_counts().nlargest(10)
top10TodasRazoes2.plot.bar(figsize=(10,5),title='Top 10 Chamadas Gerais')

### Quais municipios tem mais chamadas

In [ ]:
cidadeRazoes=df.groupby(['reason']).twp.value_counts().nlargest(10).unstack()
cidadeRazoes.plot.bar(figsize=(10,5)).legend(bbox_to_anchor=(1, 1),title='Cidades com mais chamadas')


### Analisando as variáveis pelo Histograma

In [ ]:
###Histograma das Horas
plt.style.use('ggplot')
plt.figure(figsize=(10,5))
df['hour'].fillna(df['hour'].mean(), inplace=True) #preenchendo com a média
histogram_hour = plt.hist(df['hour'], bins=8)
plt.title('Histograma Horas')
plt.show()
###Histograma dos dias da semana
plt.figure(figsize=(10,5))
df['dayOfWeekNumber'].fillna(df['dayOfWeekNumber'].mean(), inplace=True) #preenchendo com a média
histogram_Week = plt.hist(df['dayOfWeekNumber'], bins=4)
plt.title('Histograma Dias da Semana')
plt.xticks(range(len(Dicdayofweek)),[Dicdayofweek[0],Dicdayofweek[1],Dicdayofweek[2],Dicdayofweek[3],Dicdayofweek[4],Dicdayofweek[5],Dicdayofweek[6]])
plt.show()
###Histograma dos Meses
plt.figure(figsize=(10,5))
df['month'].fillna(df['month'].mean(), inplace=True) #preenchendo com a média
histogram_mes = plt.hist(df['month'], bins=5)
plt.title('Histograma Meses')
plt.show()

### Analisando chamados de Tráfego pelos meses

In [ ]:
transito = df[df["reason"]== "Traffic"].groupby("month").size()
transito
df_transito= transito.reset_index()
df_transito.columns = ['Meses','Quantidade']
df_transito.plot(x='Meses',figsize=(10,5),grid = True,title="Variação de chamadas de tráfego por meses" )

## Conclusion
Concluímos que as ferramentas apresentadas em sala, como, pandas, matplotlib, entre outras, nos dão uma ampla oportunidade de explorar os dados e mostrar graficamente informações de um dataset, fazendo com que consigamos tirar conclusões e podermos traçar alguns objetivos de acordo com as variáveis que estamos trabalhando. Neste trabalho, fizemos a análise do dataset 911, onde demonstramos o relacionamento de algumas variáveis e apresentamos alguns resultados graficamente.